In [1]:
!pip install --upgrade transformers trl huggingface_hub
!pip install rouge-score
!pip install --upgrade nltk
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
# Tokenization için NLTK veri indirme
nltk.download('punkt')
nltk.download('punkt_tab')
!pip install bert-score
!pip install inflect
!pip install -U bitsandbytes accelerate
!pip install gspread oauth2client

import gc
import torch

import transformers
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
import os
import re
import inflect
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import single_meteor_score
from bert_score import score as bert_score
import pandas as pd
import numpy as np
import pickle
import gspread
from oauth2client.service_account import ServiceAccountCredentials
pd.set_option('display.max_colwidth', None)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
  Preparing metadata (setup.py) ... done
  Created wheel for 

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2


In [2]:
df = pd.read_parquet("hf://datasets/Muadil/all_unique_cleaned_openai_summarize_comparisons_test/data/train-00000-of-00001.parquet")

In [3]:
def initialize_google_sheets():
    # Google Sheets API'ye erişim için gerekli izinleri ayarlıyoruz
    client = gspread.service_account(filename="/kaggle/input/service-account/august-oarlock-441317-t9-0a0e6dbc7072.json")

    # Google Sheet'i açıyoruz
    spreadsheet = client.open_by_url(
        'https://docs.google.com/spreadsheets/d/1E8xzMYGsCR7M9xCA2pDsfB6MhZIa6M9ZK05TUBW9iGk/edit?usp=sharing')

    # İlk sayfayı alıyoruz
    return spreadsheet.get_worksheet(0)


def sheets_to_df(worksheet):
    # Veriyi çekiyoruz ve pandas DataFrame'e çeviriyoruz
    data = worksheet.get_all_records()  # Bu, veriyi bir liste olarak alır
    df = pd.DataFrame(data)  # Veriyi DataFrame'e çeviriyoruz
    return df




In [4]:
# def write_to_google_sheets(prompts, chosens, predictions, model_name):
#     """
#     Google Sheets'e verilen verileri yazar ve modelin sütun indeksini otomatik belirler.

#     Args:
#         prompts (list): Prompt verileri.
#         chosens (list): Chosen verileri.
#         predictions (list): Prediction verileri.
#         model_name (str): Model adı (ilk satıra yazılır ve sütun indeksini belirler).
#     """
#     model_name = model_name.replace("-", "_").replace("/", "_")

#     sheet_df = sheets_to_df()
#     columns = sheet_df.columns.tolist()

#     # Eksik sütunları ekle
#     for col in ["Info", "Chosen", "Score_1"]:
#         if col not in columns:
#             sheet_df[col] = None

#     # Yeni model sütununu ekle
#     if model_name not in columns:
#         print(model_name + "column initialized")
#         sheet_df[model_name] = None
#         # Model ile ilişkili skor sütununu ekle
#         model_index = sheet_df.columns.get_loc(model_name)
#         score_col = f"Score_{model_index}"
#         sheet_df[score_col] = None

#     # Satır satır ekleme işlemi
#     for row_index, (prompt, chosen, prediction) in enumerate(zip(prompts, chosens, predictions), start=0):
#         # Eğer satır yoksa yeni bir satır ekle
#         if row_index >= len(sheet_df[model_name]):
#             sheet_df = pd.concat([sheet_df, pd.DataFrame([{col: None for col in sheet_df.columns}])], ignore_index=True)

#         # Satırdaki değerleri kontrol et ve ekle
#         if sheet_df.loc[row_index, "Info"] in [np.nan, None, ""]:
#             sheet_df.loc[row_index, "Info"] = prompt

#         if sheet_df.loc[row_index, "Chosen"] in [np.nan, None, ""]:
#             sheet_df.loc[row_index, "Chosen"] = chosen

#         if sheet_df.loc[row_index, model_name] in [np.nan, None, ""]:
#             sheet_df.loc[row_index, model_name] = prediction
#         # if pd.isna(sheet_df.loc[row_index, model_name]):
#         #     sheet_df.loc[row_index, model_name] = prediction




#     # Boş veya geçersiz verileri temizle
#     sheet_df = sheet_df.replace([float("nan"), float("inf"), float("-inf")], None)

#     # Google Sheets'e yaz
#     headers = sheet_df.columns.tolist()
#     updated_data = sheet_df.values.tolist()

#     worksheet = initialize_google_sheets()
#     worksheet.clear()
#     worksheet.append_row(headers)
#     worksheet.append_rows(updated_data)
#     print("Veri başarıyla güncellendi.")

In [5]:
def load_data_to_the_sheet(worksheet, sheet_df):
  # Boş veya geçersiz verileri temizle
  sheet_df = sheet_df.replace([float("nan"), float("inf"), float("-inf")], None)

  # Google Sheets'e yaz
  headers = sheet_df.columns.tolist()
  updated_data = sheet_df.values.tolist()

  worksheet.clear()
  worksheet.append_row(headers)
  worksheet.append_rows(updated_data)
  print("Veri başarıyla güncellendi.")

def save_the_output(sheet_df, model_name, prediction, row_index):
    

    model_name = model_name.replace("-", "_").replace("/", "_")
    # Yeni model sütununu ekle
    columns = sheet_df.columns.tolist()
    if model_name not in columns:
        print(model_name + "column initialized")
        sheet_df[model_name] = None
        # Model ile ilişkili skor sütununu ekle
        size = len(sheet_df.columns.to_list())
        score_col = f"Score_{(size-2)/2}"
        sheet_df[score_col] = None
    if sheet_df.loc[row_index, model_name] in [np.nan, None, ""]:
      sheet_df.loc[row_index, model_name] = prediction

def get_inference(model, tokenizer, text, max_token_length, model_name, device):
  text = f"System: I want you to summarize this text\nDocument: {text}\nSummary:"
  inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_token_length).to(device)
  output = model.generate(**inputs, max_new_tokens=max_token_length, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)
  prediction = tokenizer.decode(output[0], skip_special_tokens=True)
  if "instruct" in model_name.lower():
    if "Summary:" in prediction:
        prediction = prediction.split("Summary:")[1].strip()
    else:
        prediction = prediction  # Beklenmeyen durumlar için

  return prediction

In [6]:
def calculate_summary_metrics(references, predictions):
    """
    Büyük dil modellerinde başarı metriklerini hesaplamak için fonksiyon.

    Args:
        references (list): Gerçek özetlerin listesi.
        predictions (list): Model tarafından üretilen özetlerin listesi.

    Returns:
        dict: Ortalama ROUGE, METEOR, BERTScore ve MoverScore metriklerini içeren bir sözlük.
    """
    if len(references) != len(predictions):
        raise ValueError("References ve predictions aynı uzunlukta olmalı!")

    # ROUGE skorları için scorer tanımlanıyor
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1_scores, rouge2_scores, rougeL_scores = [], [], []
    meteor_scores = []

    # ROUGE ve METEOR hesaplamaları
    for ref, pred in zip(references, predictions):
        # ROUGE skorları
        rouge_scores = rouge_scorer_obj.score(ref, pred)
        rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
        rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
        rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

        # METEOR skoru (cümleleri tokenize ediyoruz)
        tokenized_ref = nltk.word_tokenize(ref)
        tokenized_pred = nltk.word_tokenize(pred)
        meteor_scores.append(single_meteor_score(tokenized_ref, tokenized_pred))

    # BERTScore hesaplaması
    P, R, F1 = bert_score(predictions, references, lang="en", rescale_with_baseline=True)
    bert_f1 = F1.mean().item()

    # MoverScore hesaplaması
    #moverscores = get_moverscore(references, predictions)

    # Ortalama skorlar
    scores = {
        'ROUGE-1': sum(rouge1_scores) / len(rouge1_scores),
        'ROUGE-2': sum(rouge2_scores) / len(rouge2_scores),
        'ROUGE-L': sum(rougeL_scores) / len(rougeL_scores),
        'METEOR': sum(meteor_scores) / len(meteor_scores),
        'BERTScore': bert_f1,
        #'MoverScore': sum(moverscores) / len(moverscores)
    }

    return scores

def add_metrics_to_dataframe(metrics_df, model_name, references, predictions):
    """
    Başarı metriklerini hesaplayarak verilen DataFrame'e ekler.

    Args:
        metrics_df (pd.DataFrame): Başarı metriklerini saklayan mevcut DataFrame.
        model_name (str): Modelin adı.
        references (list): Gerçek özetlerin listesi.
        predictions (list): Model tarafından üretilen özetlerin listesi.

    Returns:
        pd.DataFrame: Güncellenmiş metrikler tablosu.
    """
    # Metrikleri hesapla
    scores = calculate_summary_metrics(references, predictions)

    # Yeni bir satır olarak metrikleri eklemek için model adını ekle
    scores["Model Name"] = model_name
    metrics_df = pd.concat([metrics_df, pd.DataFrame([scores])], ignore_index=True)

    return metrics_df




def summarize_and_save_metrics_AutoModelForSeq2SeqLM(models, tokenizer_names, texts, references, metrics_df=None, device="cpu", max_token_length = 2048):
    """
    Modellerle özetleme yapar, başarı metriklerini hesaplar ve DataFrame'e kaydeder.

    Args:
        models (list): Özetleme yapacak model isimlerinin listesi.
        tokenizer_names (list): Model isimlerine karşılık gelen tokenizer isimlerinin listesi.
        texts (list): Özetlenecek metinlerin listesi.
        references (list): Gerçek özetlerin listesi (texts ile aynı uzunlukta olmalı).
        metrics_df (pd.DataFrame, optional): Başarı metriklerini saklayan mevcut DataFrame.
                                             None ise yeni bir DataFrame oluşturulur.
        device (str): Modeli çalıştırmak için kullanılacak cihaz ('cpu' veya 'cuda').

    Returns:
        pd.DataFrame: Güncellenmiş metrikler tablosu.
    """
    # Eğer metrics_df None ise boş bir DataFrame oluştur
    if metrics_df is None or not isinstance(metrics_df, pd.DataFrame):
        metrics_df = pd.DataFrame(columns=["Model Name", "ROUGE-1", "ROUGE-2", "ROUGE-L", "METEOR", "BERTScore"])
        print("Yeni bir metrics_df oluşturuldu.")

    # texts ve references uzunluk kontrolü
    if len(texts) != len(references):
        raise ValueError("texts ve references aynı uzunlukta olmalıdır.")


    # Quantization için BitsAndBytesConfig
    quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16, bnb_4bit_quant_type="nf4")
    # for model_idx, (model_name, tokenizer_name) in enumerate(zip(models, tokenizer_names), start=3):
    for model_name, tokenizer_name in zip(models, tokenizer_names):
        # Model ve tokenizer'ı yükle
        # model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quantization_config,
            device_map="auto"  # GPU varsa otomatik cihaz ayarı
        )
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, device_map = "auto")
        tokenizer.pad_token = tokenizer.eos_token
        print(f"{model_name} yüklendi ve çıktılar alınıyor.")

        #google sheet oku
        worksheet = initialize_google_sheets()
        sheet_df = sheets_to_df(worksheet)
        # Tahminleri oluştur
        predictions = []
        for row, text in enumerate(texts):
            # # Girişleri cihaz üzerine taşı
            # inputs = tokenizer.encode(text, return_tensors="pt", truncation=True, max_length=max_token_length).to(device)

            # # Modelden çıktı al
            # outputs = model.generate(inputs, max_length= max_token_length, num_beams=4, early_stopping=True)

            # # Çıktıyı CPU'ya taşı ve decode et
            # summary = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)
            # predictions.append(summary)

            prediction = get_inference(model, tokenizer, text, max_token_length, model_name, device)
            save_the_output(sheet_df, model_name, prediction, row)
            predictions.append(prediction)

            # Google Sheets'e yaz
        load_data_to_the_sheet(worksheet, sheet_df)

        # Başarı metriklerini DataFrame'e ekle
        metrics_df = add_metrics_to_dataframe(metrics_df, model_name, references, predictions)
        print(f"Model '{model_name}' için başarı metrikleri kaydedildi.")
        # Belleği temizle
        del model
        del tokenizer
        gc.collect()
        torch.cuda.empty_cache()
    return metrics_df




# Modellerin listesi
models = [
    "meta-llama/Llama-3.2-1B-Instruct",
    "Muadil/Llama-3.2-1B-Instruct-sum-KTO-10k_1_1ep",
]

# Tokenizer'ların listesi
tokenizer_names = [
    "meta-llama/Llama-3.2-1B-Instruct",
    "Muadil/Llama-3.2-1B-Instruct-sum-KTO-10k_1_1ep",
]
# from google.colab import userdata
# secret_value_0 = userdata.get('HF_TOKEN')
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

from huggingface_hub import login
login(token= secret_value_0)


# Başarı metriklerini saklamak için boş bir DataFrame oluştur
metrics_df = pd.DataFrame(columns=["Model Name", "ROUGE-1", "ROUGE-2", "ROUGE-L", "METEOR", "BERTScore"])

# Fonksiyonu çalıştır ve metrikleri hesapla
metrics_df = summarize_and_save_metrics_AutoModelForSeq2SeqLM(
    models=models,
    tokenizer_names=tokenizer_names,
    texts= list(df.iloc[:10]["prompt"]),
    references= list(df.iloc[:10]["chosen"]),
    metrics_df=metrics_df,
    device="cuda",  # GPU kullanımı
)

# Sonuçları görüntüle
print(metrics_df)


from IPython.display import FileLink

# Kaggle'da dosyayı Pickle dosyası olarak kaydet
file_name = "output.pkl"
metrics_df.to_pickle(file_name)

# İndirme bağlantısını oluştur
display(FileLink(file_name))

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

Muadil/Llama-3.2-1B-Instruct-sum-KTO-10k_1_1ep yüklendi ve çıktılar alınıyor.
Muadil_Llama_3.2_1B_Instruct_sum_KTO_10k_1_1epcolumn initialized
Veri başarıyla güncellendi.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model 'Muadil/Llama-3.2-1B-Instruct-sum-KTO-10k_1_1ep' için başarı metrikleri kaydedildi.
                                       Model Name   ROUGE-1   ROUGE-2  \
0                meta-llama/Llama-3.2-1B-Instruct  0.190658  0.045127   
1  Muadil/Llama-3.2-1B-Instruct-sum-KTO-10k_1_1ep  0.000386  0.000000   

    ROUGE-L    METEOR  BERTScore  
0  0.146945  0.216027   0.249753  
1  0.000386  0.000870  -0.743615  


/kaggle/working/output.pkl

In [7]:
metrics_df

,Model Name,ROUGE-1,ROUGE-2,ROUGE-L,METEOR,BERTScore
0,meta-llama/Llama-3.2-1B-Instruct,0.190658,0.045127,0.146945,0.216027,0.249753
1,Muadil/Llama-3.2-1B-Instruct-sum-KTO-10k_1_1ep,0.000386,0.000000,0.000386,0.000870,-0.743615
